In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect(database="startupdb", user='postgres', password='12345', host='127.0.0.1', port='5432')
cursor = conn.cursor()

## get_files : get a list of all song JSON files in data/song_data

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root, '*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))
    return all_files

 # Process song_data :
 ##  ETL on the dataset song_data :
 ###   ETL on a single song file and load a single record into each table to start :

## creating songs table :

In [4]:
song_files = get_files("data/song_data")
song_files[:5]

['c:\\Users\\soumm\\OneDrive\\Desktop\\Work\\Udacity\\Data Engineering\\Data Modeling with Postgres\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json',
 'c:\\Users\\soumm\\OneDrive\\Desktop\\Work\\Udacity\\Data Engineering\\Data Modeling with Postgres\\data\\song_data\\A\\A\\A\\TRAAABD128F429CF47.json',
 'c:\\Users\\soumm\\OneDrive\\Desktop\\Work\\Udacity\\Data Engineering\\Data Modeling with Postgres\\data\\song_data\\A\\A\\A\\TRAAADZ128F9348C2E.json',
 'c:\\Users\\soumm\\OneDrive\\Desktop\\Work\\Udacity\\Data Engineering\\Data Modeling with Postgres\\data\\song_data\\A\\A\\A\\TRAAAEF128F4273421.json',
 'c:\\Users\\soumm\\OneDrive\\Desktop\\Work\\Udacity\\Data Engineering\\Data Modeling with Postgres\\data\\song_data\\A\\A\\A\\TRAAAFD128F92F423A.json']

In [5]:
first = song_files[0]
df = pd.read_json(first, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


### extract songs data

In [6]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

### insert data into songs table

In [7]:
cursor.execute(songs_table_insert, song_data)
conn.commit()

## Creating artists table :

In [8]:
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


### extract artists data

In [9]:
artist_data = df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].values[0].tolist()
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

### insert data into songs table

In [10]:
cursor.execute(artists_table_insert, artist_data)
conn.commit()

# Process log_data :
##  ETL on the dataset log_data :
###  ETL on a single log file and load a single record into each table :

In [11]:
log_files = get_files("data/log_data")
first = log_files[0]
df = pd.read_json(first, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## creating songs table :

In [12]:
#Filter records by NextSong action
df = df[df['page'] == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


### extract time data

In [13]:
#Convert the ts timestamp column to datetime
# convert time to ms
time = pd.to_datetime(df["ts"], unit = 'ms')
time.head()

2   2018-11-01 21:01:46.796
4   2018-11-01 21:05:52.796
5   2018-11-01 21:08:16.796
6   2018-11-01 21:11:13.796
7   2018-11-01 21:17:33.796
Name: ts, dtype: datetime64[ns]

In [14]:
time_data = [[x, x.hour, x.day, x.week, x.month, x.year, x.dayofweek] for x in time]
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [15]:
time_df = pd.DataFrame(time_data, columns=column_labels)
time_df

,start_time,hour,day,week,month,year,weekday
0,2018-11-01 21:01:46.796,21,1,44,11,2018,3
1,2018-11-01 21:05:52.796,21,1,44,11,2018,3
2,2018-11-01 21:08:16.796,21,1,44,11,2018,3
3,2018-11-01 21:11:13.796,21,1,44,11,2018,3
4,2018-11-01 21:17:33.796,21,1,44,11,2018,3
5,2018-11-01 21:24:53.796,21,1,44,11,2018,3
6,2018-11-01 21:28:54.796,21,1,44,11,2018,3
7,2018-11-01 21:42:00.796,21,1,44,11,2018,3
8,2018-11-01 21:52:05.796,21,1,44,11,2018,3
9,2018-11-01 21:55:25.796,21,1,44,11,2018,3


### insert data into time table

In [16]:
for i, row in time_df.iterrows():
    cursor.execute(time_table_insert, list(row))
    conn.commit()

## creating users table :

In [17]:
user_df = df[["userId", "firstName", "lastName", "gender", "level"]]


### insert records into users data

In [18]:
for i, row in user_df.iterrows():
    cursor.execute(users_table_insert, row)
    conn.commit()

## creating songplays table :

In [19]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


### extract songplays data

In [25]:
for index, row in df.iterrows():

    # get song_id and artist_id from song and artist tables
    cursor.execute(song_select, (row.song, row.artist, row.length))
    results = cursor.fetchone()
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    starttime = pd.to_datetime(row.ts, unit="ms")

    # insert songplay record
    songplay_data = (index, starttime, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cursor.execute(songplays_table_insert, songplay_data)
    conn.commit()

TypeError: not all arguments converted during string formatting

# close database connection 

In [21]:
conn.close()